In [15]:
import pandas as pd
import pyarrow.parquet as pq
import math
import torch
import torch.nn as nn
import json
import torch
import json
import torch._dynamo
from torchdata.dataloader2 import DataLoader2, MultiProcessingReadingService
from torchdata.datapipes.iter import (
    FileLister,
    FileOpener,
    TFRecordLoader,
    Mapper,
    Batcher,
    Collator,
    Shuffler,
)
import torch.nn.functional as F
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import pyarrow.parquet as pq
from multiprocessing import Pool, Manager
from tqdm.notebook import tqdm_notebook
import torch
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import interp1d
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    LearningRateMonitor,
    EarlyStopping,
)


torch.set_float32_matmul_precision(
    "medium"
)  # Set the default matmul precision to medium, or high/highest?


# Read the first CSV file
dataset_train_df = pd.read_csv("train.csv")

# Read the second CSV file
dataset_supplemental_df = pd.read_csv("supplemental_metadata.csv")

# Concatenate the two dataframes
dataset_df = pd.concat([dataset_train_df, dataset_supplemental_df], ignore_index=True)

# Save the combined CSV file
# dataset_df.to_csv("train_full.csv", index=False)
dataset_df = dataset_train_df

In [16]:
# Read the first row of the DataFrame
path, sequence_id, file_id, phrase = dataset_df.iloc[0][
    ["path", "sequence_id", "file_id", "phrase"]
]
print(f"path: {path}, sequence_id: {sequence_id}, file_id: {file_id}, phrase: {phrase}")

sample_sequence_df = pq.read_table(
    f"{str(path)}",
    filters=[
        [("sequence_id", "=", sequence_id)],
    ],
).to_pandas()
print("Full sequence dataset shape is {}".format(sample_sequence_df.shape))

path: train_landmarks/5414471.parquet, sequence_id: 1816796431, file_id: 5414471, phrase: 3 creekhouse
Full sequence dataset shape is (123, 1630)


In [17]:
# Read the total amount unique files
unique_paths = dataset_df["path"].unique()

sum = unique_paths.shape[0]

print("Total number of files: {}".format(sum))

Total number of files: 68


In [18]:
LIP = [
    61,
    185,
    40,
    39,
    37,
    267,
    269,
    270,
    409,
    291,
    146,
    91,
    181,
    84,
    17,
    314,
    405,
    321,
    375,
    78,
    191,
    80,
    81,
    82,
    13,
    312,
    311,
    310,
    415,
    95,
    88,
    178,
    87,
    14,
    317,
    402,
    318,
    324,
    308,
]

FACE = (
    [f"x_face_{i}" for i in LIP]
    + [f"y_face_{i}" for i in LIP]
    + [f"z_face_{i}" for i in LIP]
)
LHAND = (
    [f"x_left_hand_{i}" for i in range(21)]
    + [f"y_left_hand_{i}" for i in range(21)]
    + [f"z_left_hand_{i}" for i in range(21)]
)
RHAND = (
    [f"x_right_hand_{i}" for i in range(21)]
    + [f"y_right_hand_{i}" for i in range(21)]
    + [f"z_right_hand_{i}" for i in range(21)]
)
POSE = (
    [f"x_pose_{i}" for i in range(33)]
    + [f"y_pose_{i}" for i in range(33)]
    + [f"z_pose_{i}" for i in range(33)]
)

SEL_COLS = FACE + LHAND + RHAND + POSE
FRAME_LEN = 384

In [19]:
# Read the existing data
with open("character_to_prediction_index.json", "r") as f:
    json_chars = json.load(f)

# Define the new entries
new_entries = [
    "<",
    ">",
    "P",
]

# Add the new entries starting from index 59, only if they don't already exist
for i, entry in enumerate(new_entries, start=59):
    if entry not in json_chars:
        json_chars[entry] = i

# Write the updated data back to the file
with open("character_to_prediction_index.json", "w") as f:
    json.dump(json_chars, f, indent=4)

start_token_idx = 59
end_token_idx = 60
pad_token_idx = 61

In [20]:
#tf.config.set_visible_devices([], "GPU")  # Disable GPU for Tensorflow
#
## Create a Manager object for the progress_queue
#manager = Manager()
#progress_queue = manager.Queue()
#
#
#def process_file(file_id):
#    file_df = dataset_df.loc[dataset_df["file_id"] == file_id]
#    path = file_df["path"].values[0]
#    parquet_df = pq.read_table(path, columns=["sequence_id"] + SEL_COLS).to_pandas()
#
#    parquet_df = parquet_df.fillna(0)
#
#    scalerFACE = StandardScaler(with_mean=True, with_std=True)
#    scalerLHAND = StandardScaler(with_mean=True, with_std=True)
#    scalerRHAND = StandardScaler(with_mean=True, with_std=True)
#    scalerPOSE = StandardScaler(with_mean=True, with_std=True)
#
#    parquet_df[FACE] = scalerFACE.fit_transform(
#        parquet_df[FACE],
#    )
#    parquet_df[LHAND] = scalerLHAND.fit_transform(
#        parquet_df[LHAND],
#    )
#    parquet_df[RHAND] = scalerRHAND.fit_transform(
#        parquet_df[RHAND],
#    )
#    parquet_df[POSE] = scalerPOSE.fit_transform(
#        parquet_df[POSE],
#    )
#
#    tf_file = f"preprocessed/{file_id}.tfrecord"
#    parquet_numpy = parquet_df.to_numpy(copy=False)
#    col_to_index = {col: i for i, col in enumerate(parquet_df.columns)}
#    LHAND_indices = [col_to_index[col] for col in LHAND]
#    RHAND_indices = [col_to_index[col] for col in RHAND]
#    buffer_size = 1000  # Adjust as needed
#    buffer = []
#
#    with tf.io.TFRecordWriter(tf_file) as file_writer:
#        for seq_id, phrase in zip(file_df["sequence_id"], file_df["phrase"]):
#            frames = parquet_numpy[parquet_df.index == seq_id]
#            progress_queue.put(
#                f"Process: {mp.current_process().name}, File: {file_id}, Sequence: {seq_id}"
#            )
#
#            current_length, num_features = frames.shape
#
#            if current_length > FRAME_LEN:
#                itp = interp1d(
#                    np.linspace(0, 1, current_length),
#                    frames,
#                    axis=0,
#                    kind="linear",
#                    fill_value="extrapolate",
#                )
#                # Generate the new index array and apply interpolation
#                new_index = np.linspace(0, 1, FRAME_LEN)
#                frames = itp(new_index)
#
#            # Calculate the number of NaN values in each hand landmark
#            r_nonan = np.sum(np.sum(np.isnan(frames[:, RHAND_indices]), axis=1) == 0)
#            l_nonan = np.sum(np.sum(np.isnan(frames[:, LHAND_indices]), axis=1) == 0)
#            no_nan = max(r_nonan, l_nonan)
#
#            if 2 * len(phrase) < no_nan:
#                features = {
#                    COL: tf.train.Feature(
#                        float_list=tf.train.FloatList(
#                            value=frames[:, col_to_index[COL]]
#                        )
#                    )
#                    for COL in SEL_COLS
#                }
#                features["phrase"] = tf.train.Feature(
#                    bytes_list=tf.train.BytesList(value=[bytes(phrase, "utf-8")])
#                )
#
#                example = tf.train.Example(features=tf.train.Features(feature=features))
#                record_bytes = example.SerializeToString()
#
#                buffer.append(record_bytes)
#                if len(buffer) == buffer_size:
#                    for record in buffer:
#                        file_writer.write(record)
#                        buffer = []
#        if buffer:
#            for record in buffer:
#                file_writer.write(record)
#
#        # gc.collect()
#
#
##cpu_count = int(mp.cpu_count() / 2)
#cpu_count = 8  # 8
#
#
#with Pool(cpu_count) as pool:
#    progress_bars = [
#        tqdm_notebook(desc=f"Process {i + 1}", unit="seq") for i in range(cpu_count)
#    ]
#
#    for result in pool.imap(
#        process_file,
#        dataset_df["file_id"].unique(),
#    ):
#        progress_updates = []
#        while not progress_queue.empty():
#            progress_updates.append(progress_queue.get())
#        for update, bar in zip(progress_updates, progress_bars):
#            bar.set_description(update)
#            bar.update()
#
#print("All parquets processed to TFRecords")

In [21]:
import os


with open("character_to_prediction_index.json", "r") as f:
    json_chars = json.load(f)  #


# Encodes phrase into a tensor of tokens
def tokenize_phrase(example):
    phrase = example["phrase"][0].decode(
        "utf-8"
    )  # Decode the byte string into a regular string
    phrase = "<" + phrase + ">"
    indices = [json_chars.get(char, json_chars.get("F")) for char in phrase]
    example["phrase"] = torch.tensor(
        indices
    )  # Replace the byte string with a list of integers
    return example


def collate_fn(batch):
    # Separate phrases and sequence lengths
    phrases = [seq.pop("phrase") for seq in batch]
    landmarks = [seq for seq in batch]

    sequence_lengths = [len(next(iter(landmark.values()))) for landmark in landmarks]
    phrase_lengths = [len(phrase) for phrase in phrases]

    # Pad sequences and phrases
    padded_batch = [
        torch.stack(
            [
                F.pad(
                    input=tensor,
                    pad=(0, FRAME_LEN - tensor.shape[0]),
                    mode="constant",
                    value=0,
                )
                for tensor in seq.values()
            ],
            dim=-1,
        )
        for seq in batch
    ]

    stacked_landmarks = torch.stack(padded_batch, dim=0)

    padded_phrases = [
        F.pad(
            input=phrase,
            pad=(0, 64 - len(phrase)),
            mode="constant",
            value=61,
        )
        for phrase in phrases
    ]

    stacked_phrases = torch.stack(padded_phrases, dim=0)

    return (
        stacked_landmarks,
        stacked_phrases,
        torch.tensor(sequence_lengths),
        torch.tensor(phrase_lengths),
    )


# Compute the split index
tf_records = dataset_df.file_id.map(
    lambda x: f"/home/jpinn/asl-fingerspelling-recognition/src/preprocessed/{x}.tfrecord"
).unique()
split_index = int(0.8 * len(tf_records))
tf_records_len = len(tf_records)

print(f"Split index: {split_index}" f"\nTotal number of TFRecords: {tf_records_len}")


def build_pipe(batch_size, drop_last, start, end, shuffle=True):
    datapipe = FileLister(tf_records[start:end])

    if shuffle:
        datapipe = Shuffler(
            datapipe, buffer_size=len(tf_records[start:end])
        )  # Shuffle the dataset

    datapipe = FileOpener(datapipe, mode="b")
    datapipe = TFRecordLoader(datapipe)
    datapipe = Mapper(datapipe, tokenize_phrase)
    datapipe = Batcher(datapipe, batch_size=batch_size, drop_last=drop_last)
    datapipe = Collator(datapipe, collate_fn=collate_fn)
    return datapipe


device = torch.device("cuda:0")

Split index: 54
Total number of TFRecords: 68


In [22]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split, Dataset
import torch
import tensorflow as tf
import random
import json
import torch.nn.functional as F


class LightningDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=256, shuffle=True):
        super().__init__()
        self.batch_size = batch_size
        self.shuffle = shuffle

    def train_dataloader(self):
        train_datapipe = build_pipe(
            batch_size=self.batch_size,
            drop_last=True,
            start=0,
            end=split_index,
            shuffle=self.shuffle,
        )
        return DataLoader2(
            datapipe=train_datapipe,
        )

    def val_dataloader(self):
        val_datapipe = build_pipe(
            batch_size=self.batch_size,
            drop_last=True,
            start=split_index,
            end=tf_records_len,
            shuffle=False,
        )
        return DataLoader2(
            datapipe=val_datapipe,
        )

In [23]:
class TokenEmbedding(nn.Module):
    def __init__(self, num_vocab=None, maxlen=None, num_hid=None):
        super(TokenEmbedding, self).__init__()
        self.num_hid = num_hid
        self.emb = nn.Embedding(num_embeddings=num_vocab, embedding_dim=num_hid)
        self.pos_emb = self.positional_encoding(maxlen - 1, num_hid)

    def forward(self, x):
        maxlen = x.size(1)
        x = x.to(torch.int64)
        x = self.emb(x)
        x = x * torch.sqrt(torch.tensor(self.num_hid, dtype=torch.float, device=device))

        return x + self.pos_emb[:maxlen, :].to(device)

    @staticmethod
    def positional_encoding(maxlen, d_model):
        position = torch.arange(0, maxlen, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        pos_encoding = torch.zeros(maxlen, d_model)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        return pos_encoding.to(device)


class LandmarkEmbedding(nn.Module):
    def __init__(self, num_hid=342, maxlen=453):
        super(LandmarkEmbedding, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=342,  # Correctly handle 342 landmark features
                out_channels=num_hid,
                kernel_size=11,
                padding=5,
            ),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(
                in_channels=num_hid, out_channels=num_hid, kernel_size=11, padding=5
            ),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(
                in_channels=num_hid, out_channels=num_hid, kernel_size=11, padding=5
            ),
            nn.ReLU(),
        )
        self.pos_emb = self.positional_encoding(maxlen, num_hid)
        self.maxlen = maxlen
        self.num_hid = num_hid

    def forward(self, x):

        # Permute the tensor to have channels as the second dimension
        x = x.permute(
            0, 2, 1
        )  # Change from [batch_size, seq_len, features] to [batch_size, features, seq_len]

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.permute(
            0, 2, 1
        )  # Optionally permute back if needed for further processing

        x = x * torch.sqrt(torch.tensor(self.num_hid, dtype=torch.float, device=device))
        return x + self.pos_emb[: x.size(1), :].to(device)

    @staticmethod
    def positional_encoding(maxlen, d_model):
        position = torch.arange(0, maxlen, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        pos_encoding = torch.zeros(maxlen, d_model)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        return pos_encoding

In [24]:
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_key_padding_mask=None):
        src2, _ = self.self_attn(src, src, src, key_padding_mask=src_key_padding_mask)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        
        src2 = self.linear2(self.dropout2(self.linear1(src)))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

In [25]:
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.multihead_attn = nn.MultiheadAttention(
            d_model, nhead, batch_first=True
        )  # Encoder-decoder attention
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm3 = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    @staticmethod
    def causal_attention_mask(batch_size, seq_len, device):
        mask = torch.triu(
            torch.ones(seq_len, seq_len, device=device), diagonal=1
        ).bool()
        return mask[None, :, :].expand(batch_size * 4, seq_len, seq_len)

    def forward(
        self, enc_out, target, src_key_padding_mask=None, tgt_key_padding_mask=None
    ):
        batch_size, seq_len, _ = target.size()

        causal_mask = self.causal_attention_mask(batch_size, seq_len, target.device)

        tgt_key_padding_mask = tgt_key_padding_mask[:, :seq_len]

        # Self-attention and layer norm
        target_att_output_only, _ = self.self_attn(
            target,
            target,
            target,
            attn_mask=causal_mask,
            key_padding_mask=tgt_key_padding_mask,
            need_weights=False,
        )
        target = self.norm1(target + self.dropout1(target_att_output_only))

        # Encoder-decoder attention and layer norm
        enc_att_output_only, _ = self.multihead_attn(
            target,
            enc_out,
            enc_out,
            key_padding_mask=src_key_padding_mask,
            need_weights=False,
        )
        target = self.norm2(target + self.dropout2(enc_att_output_only))

        # Feed forward network and layer norm
        ffn_output = self.linear2(self.dropout(F.relu(self.linear1(target))))
        target = self.norm3(target + self.dropout3(ffn_output))

        return target

In [26]:
class Transformer(nn.Module):
    def __init__(
        self,
        num_hid,
        num_head,
        num_feed_forward,
        source_maxlen,
        target_maxlen,
        num_layers_enc,
        num_layers_dec,
        num_classes,
    ):
        super(Transformer, self).__init__()
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes

        self.enc_emb = LandmarkEmbedding(num_hid=num_hid, maxlen=source_maxlen)ween
        self.dec_emb = TokenEmbedding(
            num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid
        )

        self.encoder = nn.Sequential(
            *[
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        self.decoder_layers = nn.ModuleList(
            [
                TransformerDecoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_dec)
            ]
        )

        self.classifier = nn.Linear(num_hid, num_classes)

    def decode(
        self, enc_out, target, src_key_padding_mask=None, target_padding_mask=None
    ):
        y = self.dec_emb(target)
        for i in range(self.num_layers_dec):
            y = self.decoder_layers[i](
                enc_out, y, src_key_padding_mask, target_padding_mask
            )
        return y

    def forward(
        self, source, target, src_key_padding_mask=None, tgt_key_padding_mask=None
    ):
        source = self.enc_emb(source)
        memory = source
        for layer in self.encoder:
            memory = layer(memory, src_key_padding_mask=src_key_padding_mask)
        output = self.decode(memory, target, src_key_padding_mask, tgt_key_padding_mask)
        return self.classifier(output)

    @staticmethod
    def generate_square_subsequent_mask(size):
        mask = (torch.triu(torch.ones(size, size)) == 1).float()
        mask = mask.masked_fill(mask == 0, float("-inf"))  # -inf for future positions
        return mask.unsqueeze(0)  # Add batch dimension (1)

    def generate(self, source, target_start_token_idx):
        batch_size = source.size(0)
        enc_output = self.encoder(source)

        dec_input = torch.ones(batch_size, 1, dtype=torch.long) * target_start_token_idx
        for _ in range(self.target_maxlen - 1):
            tgt_mask = self.generate_square_subsequent_mask(dec_input.size(1)).to(
                source.device
            )
            dec_out = self.decode(enc_output, dec_input, tgt_mask=tgt_mask)
            prediction = self.classifier(dec_out[:, -1])
            pred_idx = torch.argmax(prediction, dim=1, keepdim=True)
            dec_input = torch.cat([dec_input, pred_idx], dim=1)

        return dec_input.squeeze(1)

In [27]:
class TransformerModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters(config)
        self.batch_size = config["batch_size"]
        self.learning_rate = config["lr"]
        self.transformer = Transformer(
            num_hid=200,
            num_head=4,
            num_feed_forward=400,
            source_maxlen=384,
            target_maxlen=64,
            num_layers_enc=12,
            num_layers_dec=2,
            num_classes=62,
        )
        self.criterion = nn.CrossEntropyLoss(ignore_index=61)

    def forward(
        self, source, target, src_key_padding_mask=None, tgt_key_padding_mask=None
    ):
        return self.transformer(
            source,
            target,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
        )

    def create_tgt_mask(self, tgt_lengths, batch_size, seq_length):
        """Create a boolean mask for target sequences based on lengths."""
        tgt_key_padding_mask = torch.ones(
            (batch_size, seq_length), device=self.device, dtype=torch.bool
        )
        for i, length in enumerate(tgt_lengths):
            tgt_key_padding_mask[i, 0:length] = False

        return tgt_key_padding_mask

    def create_src_mask(self, src_lengths, batch_size, seq_length):
        """Create a boolean mask for source sequences based on lengths."""
        src_key_padding_mask = torch.ones(
            (batch_size, seq_length), device=self.device, dtype=torch.bool
        )
        for i, length in enumerate(src_lengths):
            src_key_padding_mask[i, 0:length] = False

        return src_key_padding_mask

    def training_step(self, batch, batch_idx):
        source, target, src_lengths, tgt_lengths = batch

        src_key_padding_mask = self.create_src_mask(
            src_lengths, source.size(0), source.size(1)
        )

        tgt_key_padding_mask = self.create_src_mask(
            tgt_lengths, target.size(0), target.size(1)
        )

        output = self.forward(
            source,
            target[:, :-1],
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
        )
        loss = self.criterion(output.transpose(1, 2), target[:, 1:])

        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, val_batch, batch_idx):
        source, target, src_lengths, tgt_lengths = val_batch

        src_key_padding_mask = self.create_src_mask(
            src_lengths, source.size(0), source.size(1)
        )

        tgt_key_padding_mask = self.create_src_mask(
            tgt_lengths, target.size(0), target.size(1)
        )

        output = self.forward(
            source,
            target[:, :-1],
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
        )
        val_loss = self.criterion(output.transpose(1, 2), target[:, 1:])

        predicted = torch.argmax(output, dim=2)
        correct = (predicted == target[:, 1:]).float()
        val_accuracy = correct.sum() / correct.numel()

        self.log(
            "val_loss",
            val_loss,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        self.log(
            "val_accuracy",
            val_accuracy,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.transformer.parameters(), lr=0.0045, weight_decay=0.08
        )

        # ADD BACK LINEAR WARMUP?
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

        # Chain scheduler/s
        scheduler = {
            "scheduler": scheduler,
            "interval": "epoch",
            "frequency": 1,
            "strict": True,
        }

        return [optimizer], [scheduler]

In [28]:
from pytorch_lightning.tuner.tuning import Tuner


def train_model():
    config = {
        "epochs": 200,
        "lr": 0.0045,
        "batch_size": 256,
    }

    # Initialize callbacks
    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="best-checkpoint",
        save_top_k=1,
        verbose=True,
        monitor="val_loss",
        mode="min",
    )
    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        patience=30,
        verbose=True,
        mode="min",
    )
    lr_monitor = LearningRateMonitor(logging_interval="step")

    # Set up Logger
    logger = TensorBoardLogger("tb_logs", name="transformer")

    # Initialize Trainer
    trainer = Trainer(
        max_epochs=config["epochs"],
        devices="auto",
        accelerator="gpu",
        callbacks=[checkpoint_callback, lr_monitor, early_stop_callback],
        enable_progress_bar=True,
        enable_model_summary=True,
        enable_checkpointing=True,
        precision="bf16-mixed",
        accumulate_grad_batches=2,
        gradient_clip_val=4,
        logger=logger,
    )

    # Create Tuner
    # tuner = Tuner(trainer)

    ## Tune Learning Rate
    # lr_find_result = tuner.lr_find(model, num_training=200, min_lr=1e-6, max_lr=1e-2)
    # if lr_find_result:
    #    model.learning_rate = lr_find_result.suggestion()

    #
    ## Start training with tuned parameters

    # Initialize the model
    model = TransformerModule(config)

    print("Optimizing the model...")
    # Compile the model using torch.compile
    optimized_model = torch.compile(model)
    
    data_module = LightningDataModule(batch_size=256, shuffle=True)

    print("Lightning Training the model...")
    trainer.fit(optimized_model, data_module)


train_model()

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jpinn/.local/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/jpinn/asl-fingerspelling-recognition/src/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Optimizing the model...
Lightning Training the model...



  | Name        | Type             | Params
-------------------------------------------------
0 | transformer | Transformer      | 6.5 M 
1 | criterion   | CrossEntropyLoss | 0     
-------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
25.964    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 5, global step 594: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 0.050 >= min_delta = 0.0. New best score: 2.417
Epoch 6, global step 693: 'val_loss' reached 2.41700 (best 2.41700), saving model to '/home/jpinn/asl-fingerspelling-recognition/src/checkpoints/best-checkpoint-v7.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
import json

# Load the idx_to_char mapping from the JSON file
with open("idx_to_char.json", "r") as f:
    idx_to_char = json.load(f)

# JSON keys are always strings, so you might need to convert them back to integers
idx_to_char = {int(k): v for k, v in idx_to_char.items()}

In [ ]:
import torch


def generate_predictions(
    model,
    dataloader,
    device,
    start_token_idx,
    end_token_idx,
    idx_to_char,
    num_samples=10,
):
    model.eval()  # Ensure the model is in evaluation mode
    preds_list = []
    ground_truth_list = []

    with torch.no_grad():  # No need to track gradients
        for batch_idx, batch in enumerate(dataloader):
            source, target = batch[0].to(device), batch[1].to(device)

            # Assuming your model has a generate function similar to model.generate
            # Adjust as necessary if your generation process is different
            preds = model.generate(source, start_token_idx).cpu().numpy()
            target = target.cpu().numpy()

            bs = source.size(0)  # Batch size

            for i in range(bs):
                target_text = "".join(
                    [idx_to_char[_] for _ in target[i, :] if _ != end_token_idx]
                )
                ground_truth_list.append(target_text)

                prediction = ""
                for idx in preds[i, :]:
                    if idx == end_token_idx:
                        break
                    prediction += idx_to_char[idx]
                preds_list.append(prediction)

            if (
                batch_idx >= num_samples - 1
            ):  # Only sample a few batches for demonstration
                break

    # Print predictions
    for i in range(min(num_samples, len(preds_list))):
        print(f"Ground Truth: {ground_truth_list[i]}")
        print(f"Prediction: {preds_list[i]}")
        print("\n~~~\n")


# Call the function with your model, validation dataloader, and other necessary parameters
generate_predictions(transformer, val_dataloader, device, 58, 59, idx_to_char)

# Model Note, Reference, Brainstorm

https://www.youtube.com/watch?v=4Bdc55j80l8

## Input Embedding Layer

The phrase must be vectorized (our case is chars)
We must add Positional Encoding to create Positional Input Embeddings

## Encoder Layer

Two sub-modules:

### Attention

#### Self-Attention

3 Distinct fully connected layers

- Query, Key, Value
- A dot product of the Query and Key matrices is computed to create a score matrix.
- The score matrix a table that dictatates how much value each word or char should be given, compared to the other words or chars in the input sequence. Higher score = more important. = more focus.
- The score matrix is normalized by dividing by the square root of the dimension of the key vectors.
- The score matrix is divided by the square root of the dimension of the key vectors which gives the scaled scores.
- The scaled scores are then passed through a softmax function to receive the attention weights.
- Multiply attention weights by value matrix to get the output vector of the self-attention layer.
- Linear layer to process.

#### Multi-headed Attention

The query, key, and value is split into N heads.

- Each vector goes through the attention layer as normal
- The output of all heads is concatenated into a single vector
- Each head is given a different representation of the input sequence, allowing the model to simultaneously attend to information from different representation subspaces.
- Each head in theory should learn to attend to different parts of the input sequence, and thus overall learn more of the input sequence.

### Residual Connection, Layer Normalization, and Feed Forward Layer

- The Multi-headed attention output vector is added to the original input vector to the sub-layer, which is called a residual connection.
- This output is then normalized by layer normalization.
- This enters a feed forward network, which is a simple 2 layer fully connected network with a ReLU activation in between.
- The output of that is added again to the original input, to be normalized again. Like before with the multi-headed attention.

## Decoder Layer

### Output Embedding Layer and Positional Encoding

- The output goes through an embedding layer to get the position embeddings.
- This enters the first multi-headed attention layer.
- The scaled scores are added to a look ahead mask, which prevents the decoder from attending to future tokens.
- This happens when the softmax makes future tokens 0, so no attention is given to them.
- All the heads are combined to create a masked output layer.

The second multi-headed attention layer has the query and key of the encoder output, and the value of the previous multi-headed attention layer output value.

A final feed forward network is applied to the output of the second multi-headed attention layer.

### Classifier, softmax, and output

The feed forward output enters a linear classifier.

Classifier output enters softmax to get the probability score of each class (char). The index of the highest probability is the predicted char.

Output is added to a list of decoded outputs until the end token is reached.

This whole structure can be stacked N layers high. The output of the final encoder, is input into all the decoder layers, who are taking the input from the previous decoder layer.


# Notes

```python
import tensorflow_addons as tfa
pbar = tfa.callbacks.TQDMProgressBar()
model.fit(…,callbacks=[pbar])
# TQDMProgressBar() also works with evaluate()
model.evaluate(…,callbacks=[pb
```

Check!

## Multiprocessing

```python
with Pool(workers) as pool:
    results = list(tqdm(pool.imap(worker,thread_list, total=len(thread_list))
                        ar])
```

Check!

## Padding strategies

1. No Padding with <EOS> token: This is the most efficient and elegant approach for Transformers.

2. Full Padding: Pad all phrases (and potentially feature sequences) to a fixed maximum length using a constant value or specific technique. This can be simpler to implement but introduces unnecessary computational overhead and potential information distortion due to large padding sections.

3. Full Padding with Masking: This combines the simplicity of full padding with the benefits of masking. While you pad all sequences to a fixed length, you apply masking during training to prevent the model from attending to the padded regions. This can be a good compromise if your model struggles with highly variable sequence lengths but you still want to avoid the downsides of excessive padding.
